In [ ]:
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
from summer2.functions.time import get_sigmoidal_interpolation_function

In [ ]:
def build_tb_model(extra_comps=[]):
    compartments = [
        "susceptible", 
        "latent_early",
        "latent_late",
        "infectious",
        "recovered",
    ] + extra_comps
    model = CompartmentalModel(
        times=[1800.0, 2024.0],
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    start_pop = {
        "susceptible": 5e7,
        "infectious": 1e3,
    }
    model.set_initial_population(start_pop)
    
    # Demographics will be important
    model.add_crude_birth_flow(
        "birth",
        Parameter("crude_birth_rate"),
        "susceptible",
    )
    model.add_universal_death_flows(
        "all_cause_mortality",
        1.0 / Parameter("life_expectancy"),
    )
    total_pop = model.request_output_for_compartments("total_population", compartments, save_results=False)
    
    # Infection / reinfection
    infection_dest = "latent_early"
    model.add_infection_frequency_flow(
        "infection", 
        Parameter("transmission_rate"),
        "susceptible", 
        infection_dest,
    )
    for source in ["latent_late", "recovered"]:
        reinfection_rate = Parameter("transmission_rate") * Parameter(f"rr_reinfection_{source}")
        model.add_infection_frequency_flow(
            f"reinfection_{source}", 
            reinfection_rate,
            source, 
            infection_dest,
        )
    
    # Latency progression
    model.add_transition_flow(
        "stabilisation",
        Parameter("stabilisation_rate"),
        "latent_early",
        "latent_late",
    )
    for progression_type in ["early", "late"]:
        progress_flow_name = f"progression_{progression_type}"
        model.add_transition_flow(
            progress_flow_name,
            Parameter(f"activation_rate_{progression_type}"),
            f"latent_{progression_type}",
            "infectious",
        )
        model.request_output_for_flow(
            progress_flow_name,
            progress_flow_name,
            save_results=False,
        )
    
    # Natural history
    model.add_transition_flow(
        "self_recovery",
        Parameter("self_recovery_rate"),
        "infectious",
        "recovered",
    )
    model.add_death_flow(
        "active_tb_death",
        Parameter("tb_death_rate"),
        "infectious",
    )
    
    # Outputs
    infectious_pop = model.request_output_for_compartments("infectious_pop", ["infectious"], save_results=False)
    model.request_function_output("prevalence", infectious_pop / total_pop * 1e5)
    inc_func = (DerivedOutput("progression_early") + DerivedOutput("progression_late")) / total_pop * 1e5
    model.request_function_output("incidence", inc_func)
    deaths = model.request_output_for_flow("deaths", "active_tb_death", save_results=False)
    death_func = deaths / total_pop * 1e5
    model.request_function_output("mortality", death_func)

    return model

In [ ]:
tb_model = build_tb_model()

In [ ]:
ragonnet_epidemics_params = {
    "stabilisation_rate": 1.0e-2,
    "activation_rate_early": 1.1e-3,
    "activation_rate_late": 5.5e-6,
}
latency_params = {k: v * 365.25 for k, v in ragonnet_epidemics_params.items()}
parameters = {
    "crude_birth_rate": 20.0 / 1000.0,
    "life_expectancy": 60.0,
    "transmission_rate": 4.0,
    "rr_reinfection_latent_late": 0.5,
    "rr_reinfection_recovered": 0.5,
    "self_recovery_rate": 0.2,
    "tb_death_rate": 0.2,
}
tb_model.run(parameters | latency_params)

In [ ]:
tb_model.get_outputs_df().plot.area()

In [ ]:
tb_model.get_derived_outputs_df().plot()

In [ ]:
# TB model with detection included
tb_detect_model = build_tb_model(["treatment"])
x_pts = [1990., 2020.]
y_pts = [0., Parameter("current_passive_detection_rate")]
passive_detection_func = get_sigmoidal_interpolation_function(x_pts, y_pts)
tb_detect_model.add_transition_flow(
    "tb_detection",
    passive_detection_func,
    "infectious",
    "treatment",
)
tb_detect_model.add_transition_flow(
    "treatment",
    1.0 / Parameter("treatment_period"),
    "treatment",
    "recovered",
)

In [ ]:
extra_params_for_detect = {
    "current_passive_detection_rate": 0.5,
    "treatment_period": 0.5,
}
tb_detect_model.run(parameters | latency_params | extra_params_for_detect)

In [ ]:
tb_detect_model.get_outputs_df().plot.area()

In [ ]:
tb_detect_model.get_derived_outputs_df().plot()